In [ ]:
# Set Jupyter to render directly to the screen
%matplotlib inline

# Import pandas and numpy for analysis
import pandas as pd
import numpy as np

# Importing the Black Scholes functions

In [ ]:
from black_scholes import call_value, put_value, call_delta, put_delta, call_vega, put_vega

In [ ]:
help(call_value)

In [ ]:
help(call_delta)

# Testing the Black-Scholes function

In [ ]:
# This is an example showing the functionality of the black_scholes module by making a plot
# of the option values and greeks for different prices of the underlying stock. You will need 
# to use the same functions, call_value, put_value, call_delta, etc. with different inputs 
# for your own trading strategy.

K = 100
T = 0.4
r = 0.0
sigma = 0.16

stock_values = range(80, 122, 2)

call_values = list()
put_values = list()
call_deltas = list()
put_deltas = list()
call_vegas = list()
put_vegas = list()

for stock_value in stock_values:
    call_values.append(call_value(stock_value, K, T, r, sigma))
    put_values.append(put_value(stock_value, K, T, r, sigma))
    call_deltas.append(call_delta(stock_value, K, T, r, sigma))
    put_deltas.append(put_delta(stock_value, K, T, r, sigma))
    call_vegas.append(call_vega(stock_value, K, T, r, sigma))
    put_vegas.append(put_vega(stock_value, K, T, r, sigma))

    
option_values = pd.DataFrame(data={'Call': call_values, 'Put': put_values}, index=stock_values)
option_values.index.name = 'Stock Value'

option_deltas = pd.DataFrame(data={'Call': call_deltas, 'Put': put_deltas}, index=stock_values)
option_deltas.index.name = 'Stock Value'

option_vegas = pd.DataFrame(data={'Call': call_vegas, 'Put': put_vegas}, index=stock_values)
option_vegas.index.name = 'Stock Value'

option_values.plot(title='Option Value - Strike {}'.format(K))
option_deltas.plot(title='Option Delta - Strike {}'.format(K))
option_vegas.plot(title='Option Vega - Strike {}'.format(K))

# Reading in the CSV file

In [ ]:
def read_data(filename):
    '''
    This function reads the .csv stored at the 'filename' location and returns three DataFrame's.
    
    The first DataFrame contains the Stock bid and ask price and volumes. It is indexed by timestamp.
    There is one level of column names, which describes whether it is a BidPrice, BidVolume, AskPrice
    or AskVolume.
    
    The second DataFrame contains the Time-to-Expiry of the options in the dataset. It is indexed by
    timestamp, and has one level of column names. There is only one column: the time to expiry.
  
    The third DataFrame contains the Option bid and ask price and volumes for different strikes. It
    is indexed by timestamp and has three levels of columns. The first level gives the strike, the second
    level gives the option type (C for Call and P for put), the third level gives whether it is a
    BidPrice, BidVolume, AskPrice or AskVolume.
    '''
    df = pd.read_csv(filename, index_col=0)
       
    time_to_expiry = df.filter(like='TimeToExpiry')
    
    stock = df.filter(like='Stock')
    stock.columns = [stock.columns.str[-5:], stock.columns.str[:-6]]
    
    options = pd.concat((df.filter(like='-P'), df.filter(like='-C')), axis=1)  
    options.columns = [options.columns.str[-3:], options.columns.str[:-4]]

    market_data = pd.concat((stock, options), axis=1)
    
    return time_to_expiry, market_data

In [ ]:
# Read the market data

filename = 'Options Arbitrage.csv'
time_to_expiry, market_data = read_data(filename)

In [ ]:
# Get a list of all instrument names including the stock, and of the options only
instrument_names = list(market_data.columns.get_level_values(0).unique())
print(instrument_names)

option_names = instrument_names[1:]
print(option_names)

# Some Examples

In [ ]:
# What is in the market_data dataframe? 
market_data.head()

In [ ]:
# What is in the time to expirty dataframe?
time_to_expiry.head()

In [ ]:
# Plot bids and asks for the stock
market_data['Stock', 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data['Stock', 'AskPrice'].head(300).plot(figsize=(20, 3))

In [ ]:
# Plot bids and asks for two of the options
print(option_names[1], option_names[4])

market_data[option_names[1], 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[1], 'AskPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[4], 'BidPrice'].head(300).plot(figsize=(20, 3))
market_data[option_names[4], 'AskPrice'].head(300).plot(figsize=(20, 3))

# Exercise

In [ ]:
# Now it's your turn. Start by calculating your own theoretical values and deltas for the options. Visualize 
# these alongside the market data, is it clear when you would want to do a trade? Enter positions when that 
# is the case, and manage your delta-risk by hedging away your delta position in the underlying stock.